In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys
module_path = r"C:\Users\Justin\Documents\GitHub\IPV_Workbench"
sys.path.insert(0, module_path)
import openpyxl
from ipv_workbench.utilities import utils
from ipv_workbench.translators import results_writers as ipv_results
import time
import os 
import pandas as pd

# Why Zero

In [18]:

project_folder = r"C:\Users\Justin\Desktop\bano_project_folder"
scenario = "AP_solar_glass_2050"


In [15]:
building_results_files

['C:\\Users\\Justin\\Desktop\\bano_project_folder\\objects\\B1393\\results\\timeseries\\CP_solar_glass_2020_string_inverter_building_level_results_hourly.csv']

In [23]:
for col in building_results_df_list[0].columns:
    print(col)

electricity_gen_bulk_building_kwh
electricity_demand_building_kwh
electricity_specific_yield_building_kwh_kwp
electricity_gen_intensity_building_kwh
electricity_gen_bulk_1393_0_5_flat_roof_kwh
surface_capacity_1393_0_5_flat_roof_kwp
surface_area_1393_0_5_flat_roof_m2
electricity_specific_yield_1393_0_5_flat_roof_kwh_kwp
electricity_gen_intensity_1393_0_5_flat_roof_kwh_m2
efficiency_1393_0_5_flat_roof_yield_irrad
irrad_bulk_1393_0_5_flat_roof_kWh
irrad_intensity_1393_0_5_flat_roof_kWh_m2
self_sufficiency_1393_0_5_flat_roof_percent
self_consumption_1393_0_5_flat_roof_percent
electricity_gen_bulk_1393_0_2_west_kwh
surface_capacity_1393_0_2_west_kwp
surface_area_1393_0_2_west_m2
electricity_specific_yield_1393_0_2_west_kwh_kwp
electricity_gen_intensity_1393_0_2_west_kwh_m2
efficiency_1393_0_2_west_yield_irrad
irrad_bulk_1393_0_2_west_kWh
irrad_intensity_1393_0_2_west_kWh_m2
self_sufficiency_1393_0_2_west_percent
self_consumption_1393_0_2_west_percent
electricity_gen_bulk_1393_0_3_flat_roof

In [21]:
for topology in ["micro_inverter","central_inverter","string_inverter"]:
    building_results_files = []
    for building in ["B1393"]:
        building_result_file = os.path.join(project_folder, 'objects', building, 'results',
                                            'timeseries',
                                            f"{scenario}_{topology}_building_level_results_hourly.csv")
        building_results_files.append(building_result_file)
    building_results_df_list = [pd.read_csv(fp, index_col="index") for fp in building_results_files]


    for direction in [("_r", "roof_tops"), ("_west_", "west_facade"), ("_east_", "east_facade"),
                        ("_north_", "north_facade"), ("_south_", "south_facade")]:
            # build col lists
            bulk_gen_buildings = []
            capacity_buildings = []
            area_buildings = []
            irrad_buildings = []
            for bldg_df in building_results_df_list:
                gen_cols = [col for col in bldg_df.columns if (f"{direction[0]}" in col) & ("gen_bulk" in col)]
                bulk_gen_buildings.append(np.sum(bldg_df[gen_cols].values, axis=1))

                capacity_cols = [col for col in bldg_df.columns if (f"{direction[0]}" in col) & ("surface_capacity_" in col)]
                capacity_buildings.append(np.sum(bldg_df[capacity_cols].values, axis=1))

                area_cols = [col for col in bldg_df.columns if (f"{direction[0]}" in col) & ("surface_area_" in col)]
                area_buildings.append(np.sum(bldg_df[area_cols].values, axis=1))

                irrad_cols = [col for col in bldg_df.columns if (f"{direction[0]}" in col) & ("irrad_bulk_" in col)]
                irrad_buildings.append(np.sum(bldg_df[irrad_cols].values, axis=1))
            
            directional_capacity = np.sum(capacity_buildings, axis=0)
            print(topology, direction, directional_capacity.min())

micro_inverter ('_r', 'roof_tops') 23.808
micro_inverter ('_west_', 'west_facade') 28.338999999999995
micro_inverter ('_east_', 'east_facade') 28.091
micro_inverter ('_north_', 'north_facade') 30.152
micro_inverter ('_south_', 'south_facade') 29.728
central_inverter ('_r', 'roof_tops') 23.808
central_inverter ('_west_', 'west_facade') 28.338999999999995
central_inverter ('_east_', 'east_facade') 28.091
central_inverter ('_north_', 'north_facade') 30.152
central_inverter ('_south_', 'south_facade') 29.728
string_inverter ('_r', 'roof_tops') 23.808
string_inverter ('_west_', 'west_facade') 28.338999999999995
string_inverter ('_east_', 'east_facade') 28.091
string_inverter ('_north_', 'north_facade') 30.152
string_inverter ('_south_', 'south_facade') 29.728


In [13]:
gen_cols = [col for col in bldg_df.columns if (f"_r_" in col) & ("gen_bulk" in col)]
gen_cols

[]

In [9]:
directional_capacity = np.sum(capacity_buildings, axis=0)
directional_capacity.min()

165.142

In [5]:
cumulative_df = ipv_results.write_cumulative_scenario_results(project_folder,
                                                                                    scenario,
                                                                                    topology,
                                                                                    building_results_df_list)

C:\Users\Justin\Documents\GitHub\IPV_Workbench\ipv_workbench\translators\results_writers.py:204: RuntimeWarning: invalid value encountered in divide
  {f"electricity_specific_yield_{direction[1]}_kwh_kwp": directional_generation / directional_capacity})
C:\Users\Justin\Documents\GitHub\IPV_Workbench\ipv_workbench\translators\results_writers.py:206: RuntimeWarning: invalid value encountered in divide
  {f"electricity_gen_intensity_{direction[1]}_kwh_m2": directional_generation / directional_installed_area})
C:\Users\Justin\Documents\GitHub\IPV_Workbench\ipv_workbench\translators\results_writers.py:213: RuntimeWarning: invalid value encountered in divide
  irrad_intensity = irrad_bulk / directional_installed_area


# Rewrite B1393 Results

In [32]:
project_folder = r"C:\Users\Justin\Desktop\bano_project_folder"
year_list = [2020, 2050, 2080]
building_list = ["B1393"]#["B1391", "B1389", "B1390", "B1360", "B1392", "B1393", "B1394", "B2494"]
all_topologies = ['micro_inverter', 'string_inverter', 'central_inverter']
log_file = os.path.join(project_folder, 'shared', 'resources', 'log_file.txt')
hourly_resolution = 3  # run every N hours (interpolate between the results at the very end)

for front_cover in ["solar_glass", "light_grey", "basic_white"][0:1]:
    for year in year_list:
        for cell_technology in ["A", "B", "C", "D"]:
            for orientation in ["P", "L"][0:1]:


                # this is key (setting the scenario)
                scenario = f"{cell_technology}{orientation}_{front_cover}_{year}"
                print(r"-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-")


                for topology in all_topologies:
                    print(scenario, topology)
                    building_results_files = []
                    for building in building_list:
                        building_result_file = os.path.join(project_folder, 'objects', building, 'results',
                                                            'timeseries',
                                                            f"{scenario}_{topology}_building_level_results_hourly.csv")
                        building_results_files.append(building_result_file)
                    building_results_df_list = [pd.read_csv(fp, index_col="index") for fp in building_results_files]
                    cumulative_df = ipv_results.write_cumulative_scenario_results(project_folder,
                                                                                    scenario,
                                                                                    topology,
                                                                                    building_results_df_list,
                                                                                    cumulative_target_folder=r"C:\Users\Justin\Desktop\bano_project_folder_test\B1393_rewrite")
                    print("Completed cumulative results.")
                    print(np.round(cumulative_df['electricity_gen_cumulative_kwh'].sum() / cumulative_df['irradiance_cumulative_kwh'].sum(),3)*100)
                    # ipv_results.write_condensed_result(project_folder,
                    #                                     building_results_df_list,
                    #                                     cumulative_df,
                    #                                     scenario,
                    #                                     topology,
                    #                                     save_to_project=False,
                    #                                     secondary_destination=r"C:\Users\Justin\Nextcloud\Teaching\22_HS\polikseni_bano\polikseni_share\condensed_simulation_results\B1393 for testing the year interpolation")


-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-
AP_solar_glass_2020 micro_inverter
['electricity_gen_bulk_1393_0_5_flat_roof_kwh', 'electricity_gen_bulk_1393_0_3_flat_roof_kwh', 'electricity_gen_bulk_1393_0_0_flat_roof_kwh', 'electricity_gen_bulk_1393_0_6_flat_roof_kwh', 'electricity_gen_bulk_1393_0_4_flat_roof_kwh']
['irrad_bulk_1393_0_5_flat_roof_kWh', 'irrad_bulk_1393_0_3_flat_roof_kWh', 'irrad_bulk_1393_0_0_flat_roof_kWh', 'irrad_bulk_1393_0_6_flat_roof_kWh', 'irrad_bulk_1393_0_4_flat_roof_kWh']
['electricity_gen_bulk_1393_0_2_west_kwh', 'electricity_gen_bulk_1393_0_12_west_kwh', 'electricity_gen_bulk_1393_0_15_west_kwh', 'electricity_gen_bulk_1393_0_9_west_kwh']
['irrad_bulk_1393_0_2_west_kWh', 'irrad_bulk_1393_0_12_west_kWh', 'irrad_bulk_1393_0_15_west_kWh', 'irrad_bulk_1393_0_9_west_kWh']
['electricity_gen_bulk_1393_0_14_east_kwh', 'electricity_gen_bulk_1393_0_7_east_kwh']
['irrad_bulk_1393_0_14_east_kWh', 'irrad_bulk_1393_0_7_east_kWh']
['electricity_gen_bulk_1393_0_1

In [27]:
cumulative_df['electricity_gen_cumulative_kwh'].sum() / cumulative_df['irradiance_cumulative_kwh'].sum()

0.06573025735365294

In [28]:
[col for col in cumulative_df.columns if "irrad_bulk_" in col]

['irrad_bulk_roof_tops_kwh',
 'irrad_bulk_west_facade_kwh',
 'irrad_bulk_east_facade_kwh',
 'irrad_bulk_north_facade_kwh',
 'irrad_bulk_south_facade_kwh']

# Why Efficiency so High

In [3]:
fp = r"C:\Users\Justin\Desktop\bano_project_folder\objects\B1393\results\timeseries\CP_solar_glass_2020_micro_inverter_building_level_results_hourly.csv"
bldg_df = pd.read_csv(fp, index_col="index")

In [20]:
def get_object_surface_area(object_hourly_results):
    area_cols = []
    for col in object_hourly_results.columns:
        if "surface_area" in col:
            area_cols.append(col)

    return object_hourly_results[area_cols].iloc[0].sum()

In [21]:
get_object_surface_area(bldg_df)

4641.78

In [19]:
a_string = "electricity_gen_bulk_1393_0_5_flat_roof_kwh"# building_results.columns.tolist()[1]
matches = ["_r", "gen_bulk"]



True

In [36]:
direction = ("_r", "roof_tops")
[col for col in bldg_df.columns if (f"{direction[0]}" in col) & ("gen_bulk" in col)]

['electricity_gen_bulk_1393_0_5_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_3_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_0_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_6_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_4_flat_roof_kwh']

In [23]:
directions = ("_r", "roof_tops")


[col for col in bldg_df.columns if all([x in col for x in [directions[0],"gen_bulk"]])]

['electricity_gen_bulk_1393_0_5_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_3_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_0_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_6_flat_roof_kwh',
 'electricity_gen_bulk_1393_0_4_flat_roof_kwh']